In [1]:
def clean_text(text):
    """
    Clean the given text.

    :param text: input text
    :type text: str
    :return: cleaned string[]
    """
    return text.strip()

In [2]:
import json
with open('../data/multidoc2dial/multidoc2dial_doc.json', 'r') as f:
    multidoc2dial_doc = json.load(f)

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
multidoc2dial_doc['doc_data']['ssa']['Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0']['spans']['1']

{'id_sp': '1',
 'tag': 'h2',
 'start_sp': 0,
 'end_sp': 61,
 'text_sp': '\n\nBenefits Planner: Survivors | Planning For Your Survivors \n',
 'title': 'Benefits Planner: Survivors | Planning For Your Survivors',
 'parent_titles': [],
 'id_sec': 't_0',
 'start_sec': 0,
 'end_sec': 61,
 'text_sec': '\n\nBenefits Planner: Survivors | Planning For Your Survivors \n'}

In [5]:
doc_sentence_train = []
doc_label_train = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        for doc_idx3 in multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['spans']:
            doc_sentence_train.append(clean_text(multidoc2dial_doc['doc_data']\
                                                 [doc_idx1][doc_idx2]['spans']\
                                                 [doc_idx3]['text_sp']))
            doc_label_train.append(doc_idx2)

In [6]:
len(doc_label_train) 

35659

In [7]:
len(set(doc_label_train))  

488

In [8]:
# for i in [1, 100, 1000, 2000, 5000]:
#     print(doc_sentence_train[i])
#     print(doc_label_train[i])
#     print('--' * 20)

In [9]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
import numpy as np
import torch
from torch.nn.functional import normalize
from sentence_transformers import SentenceTransformer

/home/grads/r/rohan.chaudhury/miniconda3/envs/nlpproj/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/grads/r/rohan.chaudhury/miniconda3/envs/nlpproj/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [10]:
model = SentenceTransformer('sentence-transformers/sentence-t5-xxl')

/home/grads/r/rohan.chaudhury/miniconda3/envs/nlpproj/lib/python3.9/site-packages/huggingface_hub/file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


In [11]:
def get_embeddings(sentence):
  return model.encode(sentence)

In [12]:
# from allennlp.common.util import pad_sequence_to_length
# from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
# from allennlp.nn.util import masked_mean, masked_softmax
# import copy

# from transformers import BertModel, T5EncoderModel
# from sentence_transformers import SentenceTransformer
# from allennlp.modules import ConditionalRandomField

# # import torch
# import math

# class CRFOutputLayer(torch.nn.Module):
#     ''' CRF output layer consisting of a linear layer and a CRF. '''
#     def __init__(self, in_dim, num_labels):
#         super(CRFOutputLayer, self).__init__()
#         self.num_labels = num_labels
#         self.classifier = torch.nn.Linear(in_dim, self.num_labels)
#         self.crf = ConditionalRandomField(self.num_labels)

#     def forward(self, x, mask, labels=None):
#         ''' x: shape: batch, max_sequence, in_dim
#             mask: shape: batch, max_sequence
#             labels: shape: batch, max_sequence
#         '''

#         batch_size, max_sequence, in_dim = x.shape

#         logits = self.classifier(x)
#         outputs = {}
#         if labels is not None:
#             log_likelihood = self.crf(logits, labels, mask)
#             loss = -log_likelihood
#             outputs["loss"] = loss
#         else:
#             best_paths = self.crf.viterbi_tags(logits, mask)
#             predicted_label = [x for x, y in best_paths]
#             predicted_label = [pad_sequence_to_length(x, desired_length=max_sequence) for x in predicted_label]
#             predicted_label = torch.tensor(predicted_label)
#             outputs["predicted_label"] = predicted_label

#             #log_denominator = self.crf._input_likelihood(logits, mask)
#             #log_numerator = self.crf._joint_likelihood(logits, predicted_label, mask)
#             #log_likelihood = log_numerator - log_denominator
#             #outputs["log_likelihood"] = log_likelihood

#         return outputs

# class CRFPerTaskOutputLayer(torch.nn.Module):
#     ''' CRF output layer consisting of a linear layer and a CRF. '''
#     def __init__(self, in_dim, tasks):
#         super(CRFPerTaskOutputLayer, self).__init__()

#         self.per_task_output = torch.nn.ModuleDict()
#         for task in tasks:
#             self.per_task_output[task.task_name] = CRFOutputLayer(in_dim=in_dim, num_labels=len(task.labels))


#     def forward(self, task, x, mask, labels=None, output_all_tasks=False):
#         ''' x: shape: batch, max_sequence, in_dim
#             mask: shape: batch, max_sequence
#             labels: shape: batch, max_sequence
#         '''
#         output = self.per_task_output[task](x, mask, labels)
#         if output_all_tasks:
#             output["task_outputs"] = []
#             assert labels is None
#             for t, task_output in self.per_task_output.items():
#                 task_result = task_output(x, mask)
#                 task_result["task"] = t
#                 output["task_outputs"].append(task_result)
#         return output

#     def to_device(self, device1, device2):
#         self.task_to_device = dict()
#         for index, task in enumerate(self.per_task_output.keys()):
#             if index % 2 == 0:
#                 self.task_to_device[task] = device1
#                 self.per_task_output[task].to(device1)
#             else:
#                 self.task_to_device[task] = device2
#                 self.per_task_output[task].to(device2)

#     def get_device(self, task):
#         return self.task_to_device[task]



# class AttentionPooling(torch.nn.Module):
#     def __init__(self, in_features, dimension_context_vector_u=200, number_context_vectors=5):
#         super(AttentionPooling, self).__init__()
#         self.dimension_context_vector_u = dimension_context_vector_u
#         self.number_context_vectors = number_context_vectors
#         self.linear1 = torch.nn.Linear(in_features=in_features, out_features=self.dimension_context_vector_u, bias=True)
#         self.linear2 = torch.nn.Linear(in_features=self.dimension_context_vector_u,
#                                        out_features=self.number_context_vectors, bias=False)

#         self.output_dim = self.number_context_vectors * in_features

#     def forward(self, tokens, mask):
#         #shape tokens: (batch_size, tokens, in_features)

#         # compute the weights
#         # shape tokens: (batch_size, tokens, dimension_context_vector_u)
#         a = self.linear1(tokens)
#         a = torch.tanh(a)
#         # shape (batch_size, tokens, number_context_vectors)
#         a = self.linear2(a)
#         # shape (batch_size, number_context_vectors, tokens)
#         a = a.transpose(1, 2)
#         a = masked_softmax(a, mask)

#         # calculate weighted sum
#         s = torch.bmm(a, tokens)
#         s = s.view(tokens.shape[0], -1)
#         return s



# class t5Embedder(torch.nn.Module):
#     def __init__(self,config):
#         super(t5Embedder, self).__init__()
#         self.bert= T5EncoderModel.from_pretrained("t5-large")
#         self.bert_trainable = config["bert_trainable"]
#         self.bert_hidden_size = self.bert.config.hidden_size
#         self.cacheable_tasks = config["cacheable_tasks"]
#         for param in self.bert.parameters():
#             param.requires_grad = self.bert_trainable

#     def forward(self, batch):
#         documents, sentences, tokens = batch["input_ids"].shape

#         if "bert_embeddings" in batch:
#             return batch["bert_embeddings"]

#         attention_mask = batch["attention_mask"].view(-1, tokens)
#         input_ids = batch["input_ids"].view(-1, tokens)

#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

#         bert_embeddings = outputs[0]


#         # if not self.bert_trainable and batch["task"] in self.cacheable_tasks:
#         #     batch["bert_embeddings"] = bert_embeddings.to("cpu")

#         return bert_embeddings


# class BertHSLN(torch.nn.Module):
#     '''
#     Model for Baseline, Sequential Transfer Learning and Multitask-Learning with all layers shared (except output layer).
#     '''
#     def __init__(self, config, tasks):
#         super(BertHSLN, self).__init__()

#         self.bert = t5Embedder(config)
#         # self.bert = BertTokenEmbedder(config)

#         # Jin et al. uses DROPOUT WITH EXPECTATION-LINEAR REGULARIZATION (see Ma et al. 2016),
#         # we use instead default dropout
#         self.dropout = torch.nn.Dropout(config["dropout"])

#         self.generic_output_layer = config.get("generic_output_layer")

#         self.lstm_hidden_size = config["word_lstm_hs"]

#         self.word_lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(input_size=self.bert.bert_hidden_size,
#                                   hidden_size=self.lstm_hidden_size,
#                                   num_layers=1, batch_first=True, bidirectional=True))

#         self.attention_pooling = AttentionPooling(2 * self.lstm_hidden_size,
#                                                   dimension_context_vector_u=config["att_pooling_dim_ctx"],
#                                                   number_context_vectors=config["att_pooling_num_ctx"])

#         self.init_sentence_enriching(config, tasks)

#         self.reinit_output_layer(tasks, config)


#     def init_sentence_enriching(self, config, tasks):
#         input_dim = self.attention_pooling.output_dim
#         print(f"Attention pooling dim: {input_dim}")
#         self.sentence_lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(input_size=input_dim,
#                                   hidden_size=self.lstm_hidden_size,
#                                   num_layers=1, batch_first=True, bidirectional=True))

#     def reinit_output_layer(self, tasks, config):
#         if config.get("without_context_enriching_transfer"):
#             self.init_sentence_enriching(config, tasks)
#         input_dim = self.lstm_hidden_size * 2

#         if self.generic_output_layer:
#             self.crf = CRFOutputLayer(in_dim=input_dim, num_labels=len(tasks[0].labels))
#         else:
#             self.crf = CRFPerTaskOutputLayer(input_dim, tasks)

#     def forward(self, batch, labels=None, output_all_tasks=False):

#         documents, sentences, tokens = batch["input_ids"].shape

#         # shape (documents*sentences, tokens, 768)
#         bert_embeddings = self.bert(batch)

#         # in Jin et al. only here dropout
#         bert_embeddings = self.dropout(bert_embeddings)

#         tokens_mask = batch["attention_mask"].view(-1, tokens)
#         # shape (documents*sentences, tokens, 2*lstm_hidden_size)
#         bert_embeddings_encoded = self.word_lstm(bert_embeddings, tokens_mask)


#         # shape (documents*sentences, pooling_out)
#         # sentence_embeddings = torch.mean(bert_embeddings_encoded, dim=1)
#         sentence_embeddings = self.attention_pooling(bert_embeddings_encoded, tokens_mask)
#         # shape: (documents, sentences, pooling_out)
#         sentence_embeddings = sentence_embeddings.view(documents, sentences, -1)
#         # in Jin et al. only here dropout
#         sentence_embeddings = self.dropout(sentence_embeddings)


#         sentence_mask = batch["sentence_mask"]

#         # shape: (documents, sentence, 2*lstm_hidden_size)
#         sentence_embeddings_encoded = self.sentence_lstm(sentence_embeddings, sentence_mask)
#         # in Jin et al. only here dropout
#         sentence_embeddings_encoded = self.dropout(sentence_embeddings_encoded)

#         if self.generic_output_layer:
#             output = self.crf(sentence_embeddings_encoded, sentence_mask, labels)
#         else:
#             output = self.crf(batch["task"], sentence_embeddings_encoded, sentence_mask, labels, output_all_tasks)


#         return output


# class BertHSLNMultiSeparateLayers(torch.nn.Module):
#     '''
#     Model Multi-Task Learning, where only certail layers are shared.
#     This class is necessary to separate the model on two GPUs.
#     '''
#     def __init__(self, config, tasks):
#         super(BertHSLNMultiSeparateLayers, self).__init__()


#         self.bert = t5Embedder(config)


#         # Jin et al. uses DROPOUT WITH EXPECTATION-LINEAR REGULARIZATION (see Ma et al. 2016),
#         # we use instead default dropout
#         self.dropout = torch.nn.Dropout(config["dropout"])

#         self.lstm_hidden_size = config["word_lstm_hs"]

#         self.word_lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(input_size=self.bert.bert_hidden_size,
#                                                              hidden_size=self.lstm_hidden_size,
#                                                              num_layers=1, batch_first=True, bidirectional=True))

#         self.attention_pooling = PerTaskGroupWrapper(
#                                         task_groups=config["attention_groups"],
#                                         create_module_func=lambda g:
#                                             AttentionPooling(2 * self.lstm_hidden_size,
#                                                   dimension_context_vector_u=config["att_pooling_dim_ctx"],
#                                                   number_context_vectors=config["att_pooling_num_ctx"])
#                                 )

#         attention_pooling_output_dim = next(iter(self.attention_pooling.per_task_mod.values())).output_dim
#         self.sentence_lstm = PerTaskGroupWrapper(
#                                     task_groups=config["context_enriching_groups"],
#                                     create_module_func=lambda g:
#                                     PytorchSeq2SeqWrapper(torch.nn.LSTM(input_size=attention_pooling_output_dim,
#                                         hidden_size=self.lstm_hidden_size,
#                                         num_layers=1, batch_first=True, bidirectional=True))
#                                     )

#         self.crf = CRFPerTaskGroupOutputLayer(self.lstm_hidden_size * 2, tasks, config["output_groups"])



#     def to_device(self, device1, device2):
#         self.bert.to(device1)
#         self.word_lstm.to(device1)
#         self.attention_pooling.to_device(device1, device2)
#         self.sentence_lstm.to_device(device1, device2)
#         self.crf.to_device(device1, device2)
#         self.device1 = device1
#         self.device2 = device2

#     def forward(self, batch, labels=None, output_all_tasks=False):
#         task_name = batch["task"]
#         documents, sentences, tokens = batch["input_ids"].shape

#         # shape (documents*sentences, tokens, 768)
#         bert_embeddings = self.bert(batch)

#         # in Jin et al. only here dropout
#         bert_embeddings = self.dropout(bert_embeddings)

#         tokens_mask = batch["attention_mask"].view(-1, tokens)
#         # shape (documents*sentences, tokens, 2*lstm_hidden_size)
#         bert_embeddings_encoded = self.word_lstm(bert_embeddings, tokens_mask)

#         # shape (documents*sentences, pooling_out)
#         # sentence_embeddings = torch.mean(bert_embeddings_encoded, dim=1)
#         device = self.attention_pooling.get_device(task_name)
#         sentence_embeddings = self.attention_pooling(task_name, bert_embeddings_encoded.to(device), tokens_mask.to(device))
#         # shape: (documents, sentences, pooling_out)
#         sentence_embeddings = sentence_embeddings.view(documents, sentences, -1)
#         # in Jin et al. only here dropout
#         sentence_embeddings = self.dropout(sentence_embeddings)

#         sentence_mask = batch["sentence_mask"]
#         # shape: (documents, sentence, 2*lstm_hidden_size)
#         device = self.sentence_lstm.get_device(task_name)
#         sentence_embeddings_encoded = self.sentence_lstm(task_name, sentence_embeddings.to(device), sentence_mask.to(device))
#         # in Jin et al. only here dropout
#         sentence_embeddings_encoded = self.dropout(sentence_embeddings_encoded)

#         device = self.crf.get_device(task_name)
#         if labels is not None:
#             labels = labels.to(device)

#         output = self.crf(task_name, sentence_embeddings_encoded.to(device), sentence_mask.to(device), labels, output_all_tasks)

#         return output

# class CRFPerTaskGroupOutputLayer(torch.nn.Module):
#     ''' CRF output layer consisting of a linear layer and a CRF. '''
#     def __init__(self, in_dim, tasks, task_groups):
#         super(CRFPerTaskGroupOutputLayer, self).__init__()

#         def get_task(name):
#             for t in tasks:
#                 if t.task_name == name:
#                     return t

#         self.crf = PerTaskGroupWrapper(
#                                         task_groups=task_groups,
#                                         create_module_func=lambda g:
#                                             # we assume same labels per group
#                                             CRFOutputLayer(in_dim=in_dim, num_labels=len(get_task(g[0]).labels))
#         )
#         self.all_tasks = [t for t in [g for g in task_groups]]


#     def forward(self, task, x, mask, labels=None, output_all_tasks=False):
#         ''' x: shape: batch, max_sequence, in_dim
#             mask: shape: batch, max_sequence
#             labels: shape: batch, max_sequence
#         '''
#         output = self.crf(task, x, mask, labels)
#         if output_all_tasks:
#             output["task_outputs"] = []
#             assert labels is None
#             for task in self.self.all_tasks:
#                 task_result = self.crf(task, x, mask, labels)
#                 task_result["task"] = task
#                 output["task_outputs"].append(task_result)
#         return output

#     def to_device(self, device1, device2):
#         self.crf.to_device(device1, device2)

#     def get_device(self, task):
#         return self.crf.get_device(task)


# class PerTaskGroupWrapper(torch.nn.Module):
#     def __init__(self, task_groups, create_module_func):
#         super(PerTaskGroupWrapper, self).__init__()

#         self.per_task_mod = torch.nn.ModuleDict()
#         for g in task_groups:
#             mod = create_module_func(g)
#             for t in g:
#                 self.per_task_mod[t] = mod

#         self.task_groups = task_groups

#     def forward(self, task_name, *args):
#         mod = self.per_task_mod[task_name]
#         return mod(*args)

#     def to_device(self, device1, device2):
#         self.task_to_device = dict()
#         for index, tasks in enumerate(self.task_groups):
#             for task in tasks:
#                 if index % 2 == 0:
#                     self.task_to_device[task] = device1
#                     self.per_task_mod[task].to(device1)
#                 else:
#                     self.task_to_device[task] = device2
#                     self.per_task_mod[task].to(device2)

#     def get_device(self, task):
#         return self.task_to_device[task]




In [13]:
# """Tokenizes the sentences with BertTokenizer as tokenisation costs some time.
# """
# import sys
# from transformers import AutoTokenizer
# import json
# from sklearn.model_selection import train_test_split
# # BERT_VOCAB = "nlpaueb/legal-bert-base-uncased"
# BERT_VOCAB = "t5-large"
# # from sentence_transformers import SentenceTransformer
# MAX_SEQ_LENGTH = 128


# def write_in_hsln_format(input,hsln_format_txt_dirpath,tokenizer):


#     final_string = ''
#     filename_sent_boundries = {}
#     for file in input:
#         file_name=file['id']
#         final_string = final_string + '###' + str(file_name) + "\n"
#         filename_sent_boundries[file_name] = {"sentence_span": []}
#         for annotation in file['annotations'][0]['result']:
#             filename_sent_boundries[file_name]['sentence_span'].append([annotation['value']['start'],annotation['value']['end']])

#             sentence_txt=annotation['value']['text']
#             sentence_label = annotation['value']['labels'][0]
#             sentence_txt = sentence_txt.replace("\r", "")
#             if sentence_txt.strip() != "":
#                 sent_tokens = tokenizer.encode(sentence_txt, add_special_tokens=True, max_length=128)
#                 sent_tokens = [str(i) for i in sent_tokens]
#                 sent_tokens_txt = " ".join(sent_tokens)
#                 final_string = final_string + sentence_label + "\t" + sent_tokens_txt + "\n"
#         final_string = final_string + "\n"
#     with open(hsln_format_txt_dirpath , "w+") as file:
#         file.write(final_string)




# def tokenize():
#     [_, train_input_json,dev_input_json,test_input_json] = sys.argv
#     tokenizer = AutoTokenizer.from_pretrained(BERT_VOCAB, do_lower_case=True)
#     train_json_format = json.load(open(train_input_json))
#     dev_json_format = json.load(open(dev_input_json))
#     test_json_format = json.load(open(test_input_json))
  
#     write_in_hsln_format(train_json_format,'datasets/pubmed-20k/train_scibert.txt',tokenizer)
#     write_in_hsln_format(dev_json_format, 'datasets/pubmed-20k/dev_scibert.txt', tokenizer)
#     write_in_hsln_format(test_json_format, 'datasets/pubmed-20k/test_scibert.txt', tokenizer)





# # tokenize()


In [14]:
# import time
# import gc
# from datetime import datetime
# from os import makedirs

# import torch

# # from eval_run import eval_and_save_metrics
# # from utils import get_device, ResultWriter, log
# # from task import pubmed_task
# # from train import SentenceClassificationTrainer
# # from models import BertHSLN
# import os

# # BERT_VOCAB = "bert-base-uncased"
# BERT_MODEL = "nlpaueb/legal-bert-base-uncased"
# # BERT_VOCAB = "bert_model/scibert_scivocab_uncased/vocab.txt"
# #BERT_MODEL = "allenai/scibert_scivocab_uncased"


# config = {
#     "bert_model": BERT_MODEL,
#     "bert_trainable": False,
#     "model": BertHSLN.__name__,
#     "cacheable_tasks": [],

#     "dropout": 0.5,
#     "word_lstm_hs": 758,
#     "att_pooling_dim_ctx": 200,
#     "att_pooling_num_ctx": 15,

#     "lr": 3e-05,
#     "lr_epoch_decay": 0.9,
#     "batch_size":  32,
#     "max_seq_length": 128,
#     "max_epochs": 20,
#     "early_stopping": 5,

# }

# MAX_DOCS = -1
# # def create_task(create_func):
# #     return create_func(train_batch_size=config["batch_size"], max_docs=MAX_DOCS)

# # def create_generic_task(task_name):
# #     return generic_task(task_name, train_batch_size=config["batch_size"], max_docs=MAX_DOCS)

# # ADAPT: Uncomment the task that has to be trained and comment all other tasks out

# # ADAPT: Uncomment the task that has to be trained and comment all other tasks out
# # task = create_task(pubmed_task)
# #task = create_task(pubmed_task_small)
# #task = create_task(nicta_task)
# #task = create_task(dri_task)
# #task = create_task(art_task)
# #task = create_task(art_task_small)
# #task = create_generic_task(GEN_DRI_TASK)
# #task = create_generic_task(GEN_PMD_TASK)
# #task = create_generic_task(GEN_NIC_TASK)
# # task = create_generic_task(GEN_ART_TASK)

# # ADAPT: Set to False if you do not want to save the best model
# save_best_models = True

# # ADAPT: provide a different device if needed
# # device = get_device(0)

# # timestamp = datetime.now().strftime("%Y-%m-%d_%H_%M_%S")

# # ADAPT: adapt the folder name of the run if necessary
# # run = f"{timestamp}_{task.task_name}_baseline"

# # -------------------------------------------

# # os.makedirs("results/complete_epoch_wise_new",exist_ok=True)
# #run_results = f'/nfs/data/sentence-classification/results/{run}'
# # run_results = f'results/{run}'
# # makedirs(run_results, exist_ok=False)

# # preload data if not already done
# # task.get_folds()

# # restarts = 1 if task.num_folds == 1 else 1
# # for restart in range(restarts):
# #     for fold_num, fold in enumerate(task.get_folds()):
# #         start = time.time()
# #         result_writer = ResultWriter(f"{run_results}/{restart}_{fold_num}_results.jsonl")
# #         result_writer.log(f"Fold {fold_num} of {task.num_folds}")
# #         result_writer.log(f"Starting training {restart} for fold {fold_num}... ")

# #         trainer = SentenceClassificationTrainer(device, config, task, result_writer)
# #         best_model = trainer.run_training_for_fold(fold_num, fold, return_best_model=save_best_models)
# #         if best_model is not None:
# #             model_path = os.path.join(run_results, f'{restart}_{fold_num}_model.pt')
# #             result_writer.log(f"saving best model to {model_path}")
# #             torch.save(best_model.state_dict(), model_path)

# #         result_writer.log(f"finished training {restart} for fold {fold_num}: {time.time() - start}")

# #         # explicitly call garbage collector so that CUDA memory is released
# #         gc.collect()

# # log("Training finished.")

# # log("Calculating metrics...")
# # eval_and_save_metrics(run_results)
# # log("Calculating metrics finished")

In [15]:
# class t5Embedder_mod(torch.nn.Module):
#     def __init__(self, model_name):
#         super(t5Embedder_mod, self).__init__()
#         self.bert= T5EncoderModel.from_pretrained(model_name)

#     def forward(self, input_ids):



#         attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)

#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

#         bert_embeddings = outputs[0]

#         bert_embeddings = bert_embeddings.mean(dim=1)
#         return bert_embeddings

In [16]:
# from transformers import T5Tokenizer, T5EncoderModel
# model_name="t5-11b"

# tokenizer = T5Tokenizer.from_pretrained(model_name, do_lower_case=True)

In [17]:
TRAIN_SIZE = len(doc_label_train)

# t5Embedder = t5Embedder_mod(model_name)
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device="cpu"
# t5Embedder.to(device)



In [18]:
# def get_embeddings(sentence):
#     input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True, max_length=128, truncation=True)).unsqueeze(0)  # Batch size 1
#     input_ids = input_ids.to(device)
#     embeddings = t5Embedder(input_ids)
#     return embeddings

In [19]:
labels = list(set(doc_label_train))
# try:
#     X=np.load('doc_spans_LaBSE_Embedding.npy')
#     y=np.load('doc_labels.npy')
# except:
#     X = []
    
#     y = []
#     progress = 0
#     for sentence, label in zip(doc_sentence_train[:TRAIN_SIZE],
#                             doc_label_train[:TRAIN_SIZE]):
#         X.append(get_embeddings(sentence))
#         y.append(labels.index(label))
#         progress += 1
#         if progress % 500 == 0:
#             print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))
#     X = np.array(X)
#     y = np.array(y)
#     print(X.shape, y.shape)
#     with open('doc_spans_LaBSE_Embedding.npy', 'wb') as f:
#         np.save(f, X)
#     with open('doc_labels.npy', 'wb') as f:
#         np.save(f, y)


In [20]:
# X = np.array(X)
# y = np.array(y)
# print(X.shape, y.shape)

In [21]:
# for i in [100]:
#     print(doc_sentence_train[i])
#     print(X[i])
#     print(doc_label_train[i])
#     print(y[i])
#     print('--' * 20)

In [22]:
# !git clone https://github.com/sadrasabouri/plda.git

In [23]:
# from plda_new import plda_new

In [24]:
# import pickle
# try:
#     with open(r"plda_clf_new_3.pkl", "rb") as input_file:
#         PLDA_classifier = pickle.load(input_file)
# except:
#     PLDA_classifier = plda_new.Classifier()
#     PLDA_classifier.fit_model(np.array(X),
#                             np.array(y))

#     with open('plda_clf_new_3.pkl', 'wb') as f:
#         pickle.dump(PLDA_classifier, f)

In [25]:

# test_queries = ["I'm looking for information regarding benefits planning, can you help me?",
#                 "I want to know about the benefits plan for survivors, can you give me more information about this?",
#                 "What are Social Security credits?",
#                 "Do you have any knowledge of Adult Disability Report? What if my spouse and I are no longer together?"]
# test_labels = ["Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
#                "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
#                "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
#                "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0"]

In [26]:
# def predict_doc(query):
#     """
#     Predict which document is matched to the given query.

#     :param query: input query
#     :type query: str (or list of strs)
#     :return: return the document name
#     """
#     query_embedding = get_embeddings(query)
#     predictions, log_p_predictions = PLDA_classifier.predict(query_embedding)
#     # print(np.argmax(log_p_predictions))
#     # print(predictions[0])
#     return [labels[i] for i in predictions[:5]]

In [27]:
# for query in test_queries:
#     print(predict_doc(query))

In [28]:
doc_sentence_train = []
doc_label_train = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        for doc_idx3 in multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['spans']:
            doc_sentence_train.append(clean_text(multidoc2dial_doc['doc_data']\
                                                 [doc_idx1][doc_idx2]['spans']\
                                                 [doc_idx3]['text_sp']))
            doc_label_train.append(doc_idx2)
labels = list(set(doc_label_train))


In [29]:
# def predict_doc_at(query, k=1):
#     """
#     Predict which document is matched to the given query.

#     :param query: input query
#     :type query: str (or list of strs)
#     :param k: number of returning docs
#     :type k: int 
#     :return: return the document name
#     """
#     query_embedding = get_embeddings(query)
#     predictions, log_p_predictions = PLDA_classifier.predict(query_embedding)
#     # print(predictions)
#     predictions = predictions[:k]
#     sum_log = np.sum(np.exp(-log_p_predictions))
#     accuracy = list(map(lambda x: np.exp(-x) / sum_log,
#                         log_p_predictions[predictions]))
#     predictions = list(map(lambda x: labels[x], predictions))
#     return accuracy, predictions

In [30]:
# for query in test_queries:
#     accs, preds = predict_doc_at(query, k=5)
#     print(accs)
#     print(preds)
#     print('-' * 20)

In [31]:
import json
with open('../data/multidoc2dial/multidoc2dial_dial_validation.json', 'r') as f:
    multidoc2dial_dial_train = json.load(f)

In [32]:

multidoc2dial_dial_train['dial_data']['dmv'][0]['turns'][0]['utterance']

'My insurance ended so what should i do'

In [33]:
multidoc2dial_dial_train['dial_data']['dmv'][0]['turns'][0]['references'][0]['doc_id']

'Top 5 DMV Mistakes and How to Avoid Them#3_0'

In [34]:
doc_sentence_test = []
doc_label_test = []
for doc_idx1 in multidoc2dial_dial_train['dial_data']:
    for dial in multidoc2dial_dial_train['dial_data'][doc_idx1]:
        for turns in dial['turns']:
            if turns['role'] == "user":
                doc_sentence_test.append(turns['utterance'])
                doc_label_test.append(turns['references'][0]['doc_id'])

In [35]:
for i in [100]:
    print(doc_sentence_test[i])
    print(doc_label_test[i])
    print('--' * 20)

what is restricted use
Conditional and restricted use licenses#3_0
----------------------------------------


In [36]:
TEST_SIZE = len(doc_sentence_test)
TEST_SIZE

4496

In [37]:
# TEST_SIZE = TEST_SIZE // 20 

In [38]:
# prec_at_500 = 0
# prec_at_100 = 0
# prec_at_50 = 0
# prec_at_10 = 0
# prec_at_5 = 0
# prec_at_1 = 0
# sample_till_now = 0
# ranks = []
# for query, act_doc in zip(doc_sentence_test[:TEST_SIZE],
#                           doc_label_test[:TEST_SIZE]):
#     accs, preds = predict_doc_at(query, k=500)
#     ranks.append(1/ (preds.index(act_doc) + 1))
#     # print(accs)
#     # print(preds)
#     # print(act_doc)
#     # print('-' * 20)
#     if act_doc == preds[0]:
#         prec_at_1 += 1
#     if act_doc in preds[:5]:
#         prec_at_5 += 1
#     if act_doc in preds[:10]:
#         prec_at_10 += 1
#     if act_doc in preds[:50]:
#         prec_at_50 += 1
#     if act_doc in preds[:100]:
#         prec_at_100 += 1
#     if act_doc in preds[:500]:
#         prec_at_500 += 1
#     sample_till_now += 1
#     if sample_till_now % 10 == 0:
#         print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
#         print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
#               format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
#                      prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
#                      prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
#                      sample_till_now))

In [39]:
label_embeddings = []
progress = 0
TRAIN_SIZE = len(labels)
for label in labels:
    label_embeddings.append(get_embeddings(label))
    progress += 1
    if progress % 50 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))

Progress Percent = 10.245901639344263%
Progress Percent = 20.491803278688526%
Progress Percent = 30.737704918032787%
Progress Percent = 40.98360655737705%
Progress Percent = 51.22950819672131%
Progress Percent = 61.47540983606557%
Progress Percent = 71.72131147540983%
Progress Percent = 81.9672131147541%
Progress Percent = 92.21311475409836%


In [40]:
# def predict_labelwise_doc_at(query, k=1):
#     """
#     Predict which document is matched to the given query.

#     :param query: input query
#     :type query: str (or list of strs)
#     :param k: number of returning docs
#     :type k: int 
#     :return: return the document name
#     """
#     query_embedding = get_embeddings(query)
#     similarities = list(map(lambda x: np.dot(x, query_embedding) /
#                             (np.linalg.norm(query_embedding) * np.linalg.norm(x)),
#                             label_embeddings))
#     similarities = np.array(similarities)
#     best_k_idx = similarities.argsort()[::-1][:k]
#     predictions = list(map(lambda x: labels[x], best_k_idx))
#     accuracy = similarities[best_k_idx]
#     return accuracy, predictions

In [41]:
# for query in test_queries:
#     accs, preds = predict_labelwise_doc_at(query, k=5)
#     print(accs)
#     print(preds)
#     print('-' * 20)

In [42]:
# prec_at_500 = 0
# prec_at_100 = 0
# prec_at_50 = 0
# prec_at_10 = 0
# prec_at_5 = 0
# prec_at_1 = 0
# sample_till_now = 0
# for query, act_doc in zip(doc_sentence_test[:TEST_SIZE],
#                           doc_label_test[:TEST_SIZE]):
#     accs, preds = predict_labelwise_doc_at(query, k=500)
#     # print(accs)
#     # print(preds)
#     # print(act_doc)
#     # print('-' * 20)
#     if act_doc == preds[0]:
#         prec_at_1 += 1
#     if act_doc in preds[:5]:
#         prec_at_5 += 1
#     if act_doc in preds[:10]:
#         prec_at_10 += 1
#     if act_doc in preds[:50]:
#         prec_at_50 += 1
#     if act_doc in preds[:100]:
#         prec_at_100 += 1
#     if act_doc in preds[:500]:
#         prec_at_500 += 1
#     sample_till_now += 1
#     if sample_till_now % 100 == 0:
#         print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
#               format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
#                      prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
#                      prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
#                      sample_till_now))

In [43]:
# prec_at_500 = 0
# prec_at_100 = 0
# prec_at_50 = 0
# prec_at_10 = 0
# prec_at_5 = 0
# prec_at_1 = 0
# rec_at_500 = 0
# rec_at_100 = 0
# rec_at_50 = 0
# rec_at_10 = 0
# rec_at_5 = 0
# rec_at_1 = 0
# sample_till_now = 0
# for query, act_doc in zip(doc_sentence_test[:TEST_SIZE],
#                           doc_label_test[:TEST_SIZE]):
#     accs, preds = predict_labelwise_doc_at(query, k=500)
#     if act_doc == preds[0]:
#         prec_at_1 += 1
#     if act_doc in preds[:5]:
#         prec_at_5 += 1
#     if act_doc in preds[:10]:
#         prec_at_10 += 1
#     if act_doc in preds[:50]:
#         prec_at_50 += 1
#     if act_doc in preds[:100]:
#         prec_at_100 += 1
#     if act_doc in preds[:500]:
#         prec_at_500 += 1
#     if act_doc != preds[0]:
#         rec_at_1 += 1
#     if act_doc not in preds[:5]:
#         rec_at_5 += 1
#     if act_doc not in preds[:10]:
#         rec_at_10 += 1
#     if act_doc not in preds[:50]:
#         rec_at_50 += 1
#     if act_doc not in preds[:100]:
#         rec_at_100 += 1
#     if act_doc not in preds[:500]:
#         rec_at_500 += 1
#     sample_till_now += 1
#     if sample_till_now % 100 == 0:
#         print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
#               format(prec_at_1 / (prec_at_1+rec_at_1), prec_at_5 / (prec_at_5+rec_at_5),
#                      prec_at_10 / (prec_at_10+rec_at_10), prec_at_50 / (prec_at_50+rec_at_50),
#                      prec_at_100 / (prec_at_1+rec_at_100), prec_at_500 / (prec_at_1+rec_at_100),
#                      sample_till_now))

In [44]:
# prec_at_500 = 0
# prec_at_100 = 0
# prec_at_50 = 0
# prec_at_10 = 0
# prec_at_5 = 0
# prec_at_1 = 0
# sample_till_now = 0
# for i in range(2, TEST_SIZE):
#     query = '.'.join(doc_sentence_test[i-2:i+1])
#     act_doc = doc_label_test[i]
#     accs, preds = predict_labelwise_doc_at(query, k=500)
#     # print(accs)
#     # print(preds)
#     # print(act_doc)
#     # print('-' * 20)
#     if act_doc == preds[0]:
#         prec_at_1 += 1
#     if act_doc in preds[:5]:
#         prec_at_5 += 1
#     if act_doc in preds[:10]:
#         prec_at_10 += 1
#     if act_doc in preds[:50]:
#         prec_at_50 += 1
#     if act_doc in preds[:100]:
#         prec_at_100 += 1
#     if act_doc in preds[:500]:
#         prec_at_500 += 1
#     sample_till_now += 1
#     if sample_till_now % 100 == 0:
#         print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
#               format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
#                      prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
#                      prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
#                      sample_till_now))

In [45]:
doc_texts_train = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        doc_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['doc_text'].strip())

In [46]:
# doc_texts_train[0]

In [47]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/sentence-t5-xxl")

In [48]:
words = set()
doc_texts_train_tokenized = []
for doc in doc_texts_train:
    tokenized_doc = [s.lower() for s in tokenizer.tokenize(doc)]
    doc_texts_train_tokenized.append(tokenized_doc) 
    words = set(tokenized_doc).union(words)
len(words)

Token indices sequence length is longer than the specified maximum sequence length for this model (1481 > 512). Running this sequence through the model will result in indexing errors


7620

In [49]:
words2IDF = {}
N_doc = len(doc_texts_train)
for i, word in enumerate(words):
    n_word = 0
    for doc in doc_texts_train_tokenized:
        if word in doc:
            n_word += 1
    words2IDF[word] = np.log(N_doc/(n_word + 1))
    if i % 1000 == 0:
        print(word, words2IDF[word])

esteem 5.497168225293202
▁publishing 5.497168225293202
▁grand 3.993090828516928
▁elect 4.804021044733257
toxic 4.580877493419047
▁contingent 4.804021044733257
cert 5.0917031171850375
▁variations 5.497168225293202


In [50]:
def calc_idf_score(sentence):
    tokenzied_sentence = [s.lower() for s in tokenizer.tokenize(sentence)]
    score = 0
    for token in tokenzied_sentence:
        if token in words2IDF:
            score += words2IDF[token]
        else:
            score += np.log(N_doc)
    return score / len(tokenzied_sentence)

In [51]:
# def predict_labelwise_doc_at_history(query_h2, query_h1, query_h0, k=1):
#     """
#     Predict which document is matched to the given query.

#     :param query: input query
#     :type query: str (or list of strs)
#     :param k: number of returning docs
#     :type k: int 
#     :return: return the document name
#     """
#     query_h2_embedding = get_embeddings(query_h2)
#     similarities2 = list(map(lambda x: np.dot(x, query_h2_embedding) /
#                             (np.linalg.norm(query_h2_embedding) * np.linalg.norm(x)),
#                             label_embeddings))
#     similarities2 = np.array(similarities2)
#     idf_score2 = calc_idf_score(query_h2)

#     query_h1_embedding = get_embeddings(query_h1)
#     similarities1 = list(map(lambda x: np.dot(x, query_h1_embedding) /
#                             (np.linalg.norm(query_h1_embedding) * np.linalg.norm(x)),
#                             label_embeddings))
#     similarities1 = np.array(similarities1)
#     idf_score1 = calc_idf_score(query_h1)

#     query_h0_embedding = get_embeddings(query_h0)
#     similarities0 = list(map(lambda x: np.dot(x, query_h0_embedding) /
#                             (np.linalg.norm(query_h0_embedding) * np.linalg.norm(x)),
#                             label_embeddings))
#     similarities0 = np.array(similarities0)
#     idf_score0 = calc_idf_score(query_h0)

#     similarities = (idf_score0 * similarities0 + \
#                     idf_score1 * similarities1 + \
#                     idf_score2 * similarities2) / \
#                     (idf_score0 + idf_score1 + idf_score2)
#     best_k_idx = similarities.argsort()[::-1][:k]
#     predictions = list(map(lambda x: labels[x], best_k_idx))
#     accuracy = similarities[best_k_idx]
#     return accuracy, predictions

In [52]:
# prec_at_500 = 0
# prec_at_100 = 0
# prec_at_50 = 0
# prec_at_10 = 0
# prec_at_5 = 0
# prec_at_1 = 0
# sample_till_now = 0
# for i in range(2, TEST_SIZE):
#     act_doc = doc_label_test[i]
#     query_h2 = doc_sentence_test[i-2]
#     query_h1 = doc_sentence_test[i-1]
#     query_h0 = doc_sentence_test[i]
#     accs, preds = predict_labelwise_doc_at_history(query_h2,
#                                                    query_h1,
#                                                    query_h0,
#                                                    k=500)
#     # print(accs)
#     # print(preds)
#     # print(act_doc)
#     # print('-' * 20)
#     if act_doc == preds[0]:
#         prec_at_1 += 1
#     if act_doc in preds[:5]:
#         prec_at_5 += 1
#     if act_doc in preds[:10]:
#         prec_at_10 += 1
#     if act_doc in preds[:50]:
#         prec_at_50 += 1
#     if act_doc in preds[:100]:
#         prec_at_100 += 1
#     if act_doc in preds[:500]:
#         prec_at_500 += 1
#     sample_till_now += 1
#     if sample_till_now % 100 == 0:
#         print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
#               format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
#                      prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
#                      prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
#                      sample_till_now))

In [53]:
import json
with open('../data/multidoc2dial/multidoc2dial_doc.json', 'r') as f:
    multidoc2dial_doc = json.load(f)

In [54]:
titles = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        titles.append(doc_idx2)

In [55]:
titles[:5]

['Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0',
 'Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#2_0',
 'Benefits Planner: Disability | How You Apply | Social Security Administration#1_0',
 'Benefits Planner: Disability | How You Apply | Social Security Administration#2_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#2_0']

In [56]:
doc_texts_train = []
title_to_domain = {}
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        doc_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['doc_text'].strip())
        title_to_domain[doc_idx2] = doc_idx1
doc_texts_train[10]

"Original Card for a Foreign Born U.S. Citizen Adult \nImportant You must present original documents or copies certified by the agency that issued them. We cannot accept photocopies or notarized copies. All documents must be current not expired. We cannot accept a receipt showing you applied for the document. \n\nWhat original documents do I need? \nCitizenship We can accept only certain documents as proof of U.S. citizenship. These include : U.S. passport ; Certificate of Naturalization N-550/N-570 ; Certificate of Citizenship N-560/N-561 ; Certificate of Report of Birth DS-1350 ; Consular Report of Birth Abroad FS-240, CRBA. Age You must present your foreign birth certificate if you have it or can get it within 10 days. If not , we will consider other documents such as your passport or a document issued by the Department of Homeland Security DHS as evidence of your age. Anyone age 12 or older requesting an original Social Security number must appear in person for an interview. We wil

In [57]:
len(doc_texts_train)

488

In [58]:
title_embeddings = []
progress = 0
TRAIN_SIZE = len(titles)
for title in titles:
    title_embeddings.append(get_embeddings(title))
    progress += 1
    if progress % 50 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))

Progress Percent = 10.245901639344263%
Progress Percent = 20.491803278688526%
Progress Percent = 30.737704918032787%
Progress Percent = 40.98360655737705%
Progress Percent = 51.22950819672131%
Progress Percent = 61.47540983606557%
Progress Percent = 71.72131147540983%
Progress Percent = 81.9672131147541%
Progress Percent = 92.21311475409836%


In [59]:
# with open('doc_title_LaBSE_Embedding.npy', 'wb') as f:
#     np.save(f, np.array(title_embeddings))

In [60]:
title_to_embeddings = {}
progress = 0
TRAIN_SIZE = len(titles)
for title in titles:
    title_to_embeddings[title] = get_embeddings(title)
    progress += 1
    if progress % 50 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))

Progress Percent = 10.245901639344263%
Progress Percent = 20.491803278688526%
Progress Percent = 30.737704918032787%
Progress Percent = 40.98360655737705%
Progress Percent = 51.22950819672131%
Progress Percent = 61.47540983606557%
Progress Percent = 71.72131147540983%
Progress Percent = 81.9672131147541%
Progress Percent = 92.21311475409836%


In [61]:
# import pickle
# with open('title_to_embeddings.pkl', 'wb') as f:
#     pickle.dump(title_to_embeddings, f)

In [62]:
# words2IDF = {}
# N_doc = len(doc_texts_train)
# for i, word in enumerate(words):
#     n_word = 0
#     for doc in doc_texts_train_tokenized:
#         if word in doc:
#             n_word += 1
#     words2IDF[word] = np.log(N_doc / (n_word + 1))
#     if i % 1000 == 0:
#         print(word, words2IDF[word])

In [63]:
len(words2IDF)

7620

In [64]:
# def calc_idf_score(sentence):
#     tokenzied_sentence = [s.lower() for s in tokenizer.tokenize(sentence)]
#     score = 0
#     for token in tokenzied_sentence:
#         if token in words2IDF:
#             score += words2IDF[token]
#         else:
#             score += np.log(N_doc)
#     return score / len(tokenzied_sentence)

In [65]:
import pickle
with open('IDFs.pkl', 'wb') as f:
    pickle.dump(words2IDF, f)

In [66]:
# def predict_labelwise_doc_at_history(queries, k=1):
#     similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
#     coef_sum = 0
#     for query in queries:
#         query_embd = get_embeddings(query)
#         query_sim = list(map(lambda x: np.dot(x, query_embd) /
#                             (np.linalg.norm(query_embd) * np.linalg.norm(x)),
#                             title_embeddings))
#         query_sim = np.array(query_sim)

#         coef = calc_idf_score(query)
#         coef_sum += coef
#         similarities += coef * query_sim

#     similarities = similarities / coef_sum
#     best_k_idx = similarities.argsort()[::-1][:k]
#     predictions = list(map(lambda x: titles[x], best_k_idx))
#     accuracy = similarities[best_k_idx]
#     return accuracy, predictions

In [67]:
# def predict_labelwise_doc_at_history_ordered(queries, k=1):
#     similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
#     coef_sum = 0
#     for i, query in enumerate(queries):
#         query_embd = get_embeddings(query)
#         query_sim = list(map(lambda x: np.dot(x, query_embd) /
#                             (np.linalg.norm(query_embd) * np.linalg.norm(x)),
#                             title_embeddings))
#         query_sim = np.array(query_sim)

#         coef = 2**(-i) * calc_idf_score(query)
#         coef_sum += coef
#         similarities += coef * query_sim

#     similarities = similarities / coef_sum
#     best_k_idx = similarities.argsort()[::-1][:k]
#     predictions = list(map(lambda x: titles[x], best_k_idx))
#     accuracy = similarities[best_k_idx]
#     return accuracy, predictions

In [68]:
# def predict_labelwise_doc_at_history_ordered_softmaxed(queries, k=1):
#     similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
#     coefs = []
#     sims = []
#     for i, query in enumerate(queries):
#         query_embd = get_embeddings(query)
#         query_sim = list(map(lambda x: np.dot(x, query_embd) /
#                             (np.linalg.norm(query_embd) * np.linalg.norm(x)),
#                             title_embeddings))
#         sims.append(np.array(query_sim))
#         coefs.append(2**(-i) * calc_idf_score(query))
    
#     # Softmax:
#     coefs = np.array(list(map(lambda x: np.exp(-x), coefs)))
#     coefs /= coefs.sum()
#     coefs = list(coefs)

#     for coef, sim in zip(coefs, sims):
#         similarities += coef * sim
#     best_k_idx = similarities.argsort()[::-1][:k]
#     predictions = list(map(lambda x: titles[x], best_k_idx))
#     accuracy = similarities[best_k_idx]
#     return accuracy, predictions

In [69]:
# def predict_labelwise_doc_at_history_selfatt(queries, k=1):
#     similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
#     coef_sum = 0
#     query0_embd = get_embeddings(queries[0])
#     for query in queries:
#         query_embd = get_embeddings(query)
#         query_sim = list(map(lambda x: np.dot(x, query_embd) /
#                             (np.linalg.norm(query_embd) * np.linalg.norm(x)),
#                             title_embeddings))
#         query_sim = np.array(query_sim)

#         coef = calc_idf_score(query) * np.dot(query0_embd, query_embd) / (np.linalg.norm(query_embd) * np.linalg.norm(query0_embd))
#         coef_sum += coef
#         similarities += coef * query_sim

#     similarities = similarities / coef_sum
#     best_k_idx = similarities.argsort()[::-1][:k]
#     predictions = list(map(lambda x: titles[x], best_k_idx))
#     accuracy = similarities[best_k_idx]
#     return accuracy, predictions

In [70]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(strip_accents=None,
                                 analyzer='char',
                                 ngram_range=(2, 8),
                                 norm='l2',
                                 use_idf=True,
                                 smooth_idf=True,
                                 stop_words='english')
tfidf_wm = tfidfVectorizer.fit_transform(doc_texts_train)

In [71]:
import pickle
with open('tfidfVectorizer.pkl', 'wb') as f:
    pickle.dump(tfidfVectorizer, f)

with open('tfidf_wm.pkl', 'wb') as f:
    pickle.dump(tfidf_wm, f)

In [72]:
def predict_DR_TEIT(queries, k=1, alpha=10):
    idf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    tfidf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    query0_embd = get_embeddings(queries[0])
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        # print(query_embd.shape)
        # query_sim = list(map(lambda x: np.dot(x, query_embd) /
        #                     (np.linalg.norm(query_embd) * np.linalg.norm(x)),
        #                     title_embeddings))
        query_sim = list(map(lambda x: cosine_similarity(x.reshape(1, -1), query_embd.reshape(1, -1))[0][0], title_embeddings))
        # query_sim = list(map(lambda x: cosine_similarity(x.detach().numpy().reshape(1, -1), query_embd.detach().numpy().reshape(1, -1))[0][0], title_embeddings))
        query_sim = np.array(query_sim)
        coef= 2**(-i) * calc_idf_score(query)
        # coef = np.exp(-i) * calc_idf_score(query)
        # coef = calc_idf_score(query) * np.exp(cosine_similarity(query0_embd.reshape(1, -1), query_embd.reshape(1, -1))[0][0])
        coef_sum += coef

        idf_score += coef * query_sim
        tfidf_score += coef * np.squeeze(np.asarray(tfidf_wm @ tfidfVectorizer.transform([query]).todense().T))

    scores = (idf_score + alpha * tfidf_score) / coef_sum
    best_k_idx = scores.argsort()[::-1][:k]
    scores = scores[best_k_idx]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    return (scores, predictions)

In [73]:
test_queries = ["I'm looking for information regarding benefits planning, can you help me?",
                "I want to know about the benefits plan for survivors, can you give me more information about this?",
                "What are Social Security credits?"]
test_labels = ["Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
               "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
               "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0"]

In [74]:

import json
with open('../data/multidoc2dial/multidoc2dial_dial_validation.json', 'r') as f:
    multidoc2dial_dial_train = json.load(f)

In [75]:
doc_sentence_test = []
doc_label_test = []
for doc_idx1 in multidoc2dial_dial_train['dial_data']:
    for dial in multidoc2dial_dial_train['dial_data'][doc_idx1]:
        for turns in dial['turns']:
            if turns['role'] == "user":
                doc_sentence_test.append(turns['utterance'])
                doc_label_test.append(turns['references'][0]['doc_id'])

In [76]:
# TEST_SIZE = len(doc_sentence_test)
# TEST_SIZE

In [77]:
accs, preds = predict_DR_TEIT([test_queries[2],
                                               test_queries[1],
                                               test_queries[0]],
                                               k=5)
print(accs)
print(preds)
print('-' * 20)

[2.80636926 2.80438331 2.73998604 2.64887847 2.44721318]
['Benefits Planner | Social Security Credits and Benefit Eligibility | SSA#2_0', 'Benefits Planner | Social Security Credits and Benefit Eligibility | SSA#1_0', 'Learn About Retirement Benefits | SSA#1_0', 'Manage Your Retirement Benefits | SSA#1_0', 'Benefits Planner | Social Security Administration#1_0']
--------------------


In [78]:
# prec_at_500 = 0
# prec_at_100 = 0
# prec_at_50 = 0
# prec_at_10 = 0
# prec_at_5 = 0
# prec_at_1 = 0
# sample_till_now = 0
# ranks = []
# for i in range(2, TEST_SIZE):
#     act_doc = doc_label_test[i]
#     accs, preds = predict_labelwise_doc_at_history([doc_sentence_test[i],
#                                                    doc_sentence_test[i-1],
#                                                    doc_sentence_test[i-2]],
#                                                    k=500)
#     ranks.append(1 / (preds.index(act_doc) + 1))
#     if act_doc == preds[0]:
#         prec_at_1 += 1
#     if act_doc in preds[:5]:
#         prec_at_5 += 1
#     if act_doc in preds[:10]:
#         prec_at_10 += 1
#     if act_doc in preds[:50]:
#         prec_at_50 += 1
#     if act_doc in preds[:100]:
#         prec_at_100 += 1
#     if act_doc in preds[:500]:
#         prec_at_500 += 1
#     sample_till_now += 1
#     if sample_till_now % 100 == 0:
#         print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
#         print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
#               format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
#                      prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
#                      prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
#                      sample_till_now))

In [79]:
# accs, preds = predict_labelwise_doc_at_history_ordered([test_queries[2],
#                                                         test_queries[1],
#                                                         test_queries[0]],
#                                                         k=5)
# print(accs)
# print(preds)
# print('-' * 20)

In [80]:
# prec_at_500 = 0
# prec_at_100 = 0
# prec_at_50 = 0
# prec_at_10 = 0
# prec_at_5 = 0
# prec_at_1 = 0
# sample_till_now = 0
# ranks = []
# for i in range(2, TEST_SIZE):
#     act_doc = doc_label_test[i]
#     accs, preds = predict_labelwise_doc_at_history_ordered([doc_sentence_test[i],
#                                                             doc_sentence_test[i-1],
#                                                             doc_sentence_test[i-2]],
#                                                             k=500)
#     ranks.append(1 / (preds.index(act_doc) + 1))
#     if act_doc == preds[0]:
#         prec_at_1 += 1
#     if act_doc in preds[:5]:
#         prec_at_5 += 1
#     if act_doc in preds[:10]:
#         prec_at_10 += 1
#     if act_doc in preds[:50]:
#         prec_at_50 += 1
#     if act_doc in preds[:100]:
#         prec_at_100 += 1
#     if act_doc in preds[:500]:
#         prec_at_500 += 1
#     sample_till_now += 1
#     if sample_till_now % 100 == 0:
#         print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
#         print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
#               format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
#                      prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
#                      prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
#                      sample_till_now))

In [81]:
# accs, preds = predict_labelwise_doc_at_history_ordered_softmaxed([test_queries[2],
#                                                         test_queries[1],
#                                                         test_queries[0]],
#                                                         k=5)
# print(accs)
# print(preds)
# print('-' * 20)

In [82]:
# prec_at_500 = 0
# prec_at_100 = 0
# prec_at_50 = 0
# prec_at_10 = 0
# prec_at_5 = 0
# prec_at_1 = 0
# sample_till_now = 0
# ranks = []
# for i in range(2, TEST_SIZE):
#     act_doc = doc_label_test[i]
#     accs, preds = predict_labelwise_doc_at_history_ordered_softmaxed([doc_sentence_test[i],
#                                                             doc_sentence_test[i-1],
#                                                             doc_sentence_test[i-2]],
#                                                             k=500)
#     ranks.append(1 / (preds.index(act_doc) + 1))
#     if act_doc == preds[0]:
#         prec_at_1 += 1
#     if act_doc in preds[:5]:
#         prec_at_5 += 1
#     if act_doc in preds[:10]:
#         prec_at_10 += 1
#     if act_doc in preds[:50]:
#         prec_at_50 += 1
#     if act_doc in preds[:100]:
#         prec_at_100 += 1
#     if act_doc in preds[:500]:
#         prec_at_500 += 1
#     sample_till_now += 1
#     if sample_till_now % 100 == 0:
#         print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
#         print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
#               format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
#                      prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
#                      prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
#                      sample_till_now))

In [83]:
# accs, preds = predict_labelwise_doc_at_history_selfatt([test_queries[2],
#                                                         test_queries[1],
#                                                         test_queries[0]],
#                                                         k=5)
# print(accs)
# print(preds)
# print('-' * 20)

In [84]:
# prec_at_500 = 0
# prec_at_100 = 0
# prec_at_50 = 0
# prec_at_10 = 0
# prec_at_5 = 0
# prec_at_1 = 0
# sample_till_now = 0
# ranks = []
# for i in range(2, TEST_SIZE):
#     act_doc = doc_label_test[i]
#     accs, preds = predict_labelwise_doc_at_history_selfatt([doc_sentence_test[i],
#                                                             doc_sentence_test[i-1],
#                                                             doc_sentence_test[i-2]],
#                                                             k=500)
#     ranks.append(1 / (preds.index(act_doc) + 1))
#     if act_doc == preds[0]:
#         prec_at_1 += 1
#     if act_doc in preds[:5]:
#         prec_at_5 += 1
#     if act_doc in preds[:10]:
#         prec_at_10 += 1
#     if act_doc in preds[:50]:
#         prec_at_50 += 1
#     if act_doc in preds[:100]:
#         prec_at_100 += 1
#     if act_doc in preds[:500]:
#         prec_at_500 += 1
#     sample_till_now += 1
#     if sample_till_now % 100 == 0:
#         print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
#         print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
#               format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
#                      prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
#                      prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
#                      sample_till_now))

In [85]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_DR_TEIT([doc_sentence_test[i],
                                   doc_sentence_test[i-1],
                                   doc_sentence_test[i-2]],
                                   k=500,
                                   alpha=10)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.6905196672725737, var=0.1393508103248923
Prec@(1) = 0.56 | Prec@(5) = 0.84 | Prec@(10) = 0.87 | Prec@(50) = 0.97 | Prec@(100) = 0.98 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.6472168825262932, var=0.13998292736612034
Prec@(1) = 0.5 | Prec@(5) = 0.855 | Prec@(10) = 0.9 | Prec@(50) = 0.97 | Prec@(100) = 0.975 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.6788619253813933, var=0.13763722659455713
Prec@(1) = 0.5433333333333333 | Prec@(5) = 0.8633333333333333 | Prec@(10) = 0.9066666666666666 | Prec@(50) = 0.9733333333333334 | Prec@(100) = 0.9766666666666667 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.6955904072776633, var=0.13806540773714265
Prec@(1) = 0.5725 | Prec@(5) = 0.8675 | Prec@(10) = 0.905 | Prec@(50) = 0.9675 | Prec@(100) = 0.9775 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.6927526918888125, var=0.1400697360121619
Prec@(1) = 0.572 | Prec@(5) = 0.856 | Prec@(10) = 0.904 | Prec@(50) = 0.968 | Prec@(100) = 0.982 | Prec@(

In [ ]:
def _load_doc_data_rc():
    
    doc_filepath = os.path.join("../data", "multidoc2dial/multidoc2dial_doc.json")
    with open(doc_filepath, encoding="utf-8") as f:
        data = json.load(f)["doc_data"]
    return data

In [72]:

def _get_answers_rc(references, spans, doc_text):
    """Obtain the grounding annotation for a given dialogue turn"""
    if not references:
        return []
    start, end = -1, -1
    ls_sp = []
    for ele in references:
        id_sp = ele["id_sp"]
        start_sp, end_sp = spans[id_sp]["start_sp"], spans[id_sp]["end_sp"]
        if start == -1 or start > start_sp:
            start = start_sp
        if end < end_sp:
            end = end_sp
        ls_sp.append(doc_text[start_sp:end_sp])
    answer = {"text": doc_text[start:end], "answer_start": start}
    return [answer]

In [73]:


doc_data = _load_doc_data_rc()
d_doc_data = {}
for domain, d_doc in doc_data.items():
    for doc_id, data in d_doc.items():
        d_doc_data[doc_id] = data
qa_all = []
with open("../data/multidoc2dial/multidoc2dial_dial_train.json", encoding="utf-8") as f:
    dial_data = json.load(f)["dial_data"]
    for domain, dialogues in dial_data.items():
        for dial in dialogues:
            all_prev_utterances = []
            for idx, turn in enumerate(dial["turns"]):
                doc_id = turn["references"][0]["doc_id"]
                doc = d_doc_data[doc_id]
                utterance_line = turn["utterance"].replace("\n", " ").replace("\t", " ")
                all_prev_utterances.append("{}: {}".format(turn["role"], utterance_line))
                if turn["role"] == "agent":
                    continue
                if idx + 1 < len(dial["turns"]):
                    if (
                        dial["turns"][idx + 1]["role"] == "agent"
                        and dial["turns"][idx + 1]["da"] != "respond_no_solution"
                    ):
                        turn_to_predict = dial["turns"][idx + 1]
                    else:
                        continue
                else:
                    continue
                question_str1=utterance_line
                question_str = utterance_line + "[SEP]" + "||".join(reversed(all_prev_utterances[:-1]))
                id_ = "{}_{}".format(dial["dial_id"], turn["turn_id"])
                qa = {
                    "id": id_,
                    "title": doc_id,
                    # "context": doc["doc_text"],
                    "question": question_str,
                    "da": turn["da"],
                    "answers": _get_answers_rc(
                        turn_to_predict["references"], doc["spans"], doc["doc_text"]
                    ),
                    "utterance": turn_to_predict["utterance"],
                    "domain": domain,
                    "short_question" : question_str1,
                    "context": "||".join(all_prev_utterances[:-1])
                }
                qa_all.append(qa)
                

In [74]:
print(qa_all[:5])

[{'id': '8df07b7a98990db27c395cb1f68a962e_1', 'title': 'Top 5 DMV Mistakes and How to Avoid Them#3_0', 'question': 'Hello, I forgot o update my address, can you help me with that?[SEP]', 'da': 'query_condition', 'answers': [{'text': 'you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ', 'answer_start': 346}], 'utterance': 'hi, you have to report any change of address to DMV within 10 days after moving. You should do this both for the address associated with your license and all the addresses associated with all your vehicles.', 'domain': 'dmv', 'short_question': 'Hello, I forgot o update my address, can you help me with that?', 'context': ''}, {'id': '8df07b7a98990db27c395cb1f68a962e_3', 'title': 'Top 5 DMV Mistakes and How to Avoid Them#3_0', 'question': 'Can I do my DMV transactions online?[SEP]agent: hi, you have to

In [75]:
print(doc_sentence_test[:5])

['Hello, I forgot o update my address, can you help me with that?', 'Can I do my DMV transactions online?', "You've got it. Another query about DMV. What happens if you seek a hearing with the DMV?", 'Besides that, will I receive a notice?', 'If you submit the affidavit?']


In [76]:
# words2IDF = {}
# N_doc = len(doc_texts_train)
# for i, word in enumerate(words):
#     n_word = 0
#     for doc in doc_texts_train_tokenized:
#         if word in doc:
#             n_word += 1
#     words2IDF[word] = np.log(N_doc/(n_word + 1))
#     if i % 1000 == 0:
#         print(word, words2IDF[word])

In [77]:
# def calc_idf_score(sentence):
#     tokenzied_sentence = [s.lower() for s in tokenizer.tokenize(sentence)]
#     score = 0
#     for token in tokenzied_sentence:
#         if token in words2IDF:
#             score += words2IDF[token]
#         else:
#             score += np.log(N_doc)
#     return score / len(tokenzied_sentence)

In [78]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidfVectorizer = TfidfVectorizer(strip_accents=None,
#                                  analyzer='char',
#                                  ngram_range=(2, 8),
#                                  norm='l2',
#                                  use_idf=True,
#                                  smooth_idf=True,
#                                  stop_words='english')
# tfidf_wm = tfidfVectorizer.fit_transform(doc_texts_train)

In [79]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
model_name = "deepset/roberta-large-squad2"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=device)
print(device)

cuda:0


In [80]:
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_answer=""
    l_question = qa_all[i]["short_question"]
    l_context = qa_all[i]["context"]
    for answers in qa_all[i]["answers"]:
        act_answer = act_answer + " " + answers["text"]
    accs, preds = predict_DR_TEIT([doc_sentence_test[i],
                                   doc_sentence_test[i-1],
                                   doc_sentence_test[i-2]],
                                   k=50,
                                   alpha=10)
    individual_texts_train = []
    # for pred in preds:
    #     for doc_idx2 in multidoc2dial_doc['doc_data'][pred]:
    #         individual_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
    #                                         [doc_idx2]['doc_text'].strip())
    all_text=[]
    short_question=doc_sentence_test[i]
    for pred in preds[:20]:
        # print(pred)
        # print(d_doc_data[pred]["spans"])
        all_text.append(d_doc_data[pred]["doc_text"])
        k=1
        string_int="1"
        while(k==1):
            try:
                individual_texts_train.append(d_doc_data[pred]["spans"][string_int]["text_sp"].strip())
                string_int = str(int(string_int)+1)
            except:
                k=0
        # for t_spans in d_doc_data[pred]["spans"]:
        #     print(t_spans)
        #     individual_texts_train.append(d_doc_data[pred]["doc_text"][int(t_spans["start_sp"]):int(t_spans["end_sp"])].strip())
    
    # all_text = "\n".join(all_text)+"\n"+l_context

    # QA_input = {
    #       'question': long_question,
    #       'context': all_text
    #       }
    scores=[]
    results=[]
    for texts in all_text:
        QA_input2 = {
            'question': short_question,
            'context': texts + "\n" + l_context + "||"
            }
        output = nlp(QA_input2, topk=10)
        
        for out in output:
            results.append(out["answer"])
            scores.append(out["score"])
    res=[x for _,x in sorted(zip(scores,results), reverse=True)]
    res=results[:10]

    # res = nlp(QA_input, max_answer_len=100,  handle_impossible_answer=True, max_question_len=100, allow_long_answers=True, align_to_words=True)
    # res2= nlp(QA_input2, max_answer_len=100, handle_impossible_answer=True, max_question_len=100, allow_long_answers=True, align_to_words=True)
    
    prel_ans=res
    # prel_ans2=res2["answer"]
    print("Context: ", l_context+ "||")
    print("Actual Question: ", l_question)
    print("Short Question: ", short_question)
    print("roberta ans:", prel_ans)
    print("Actual Answer: ", act_answer)
    print("\n\n")
    # individual_texts_train = list(set(individual_texts_train))
    # cosine_sim=[]
    # query_embedding = get_embeddings(prel_ans).reshape(1,-1)
    # for i in range(len(individual_texts_train)):
    #     cosine_sim.append(cosine_similarity(query_embedding, get_embeddings(individual_texts_train[i]).reshape(1,-1))[0][0])
    # individual_texts_train = [x for _,x in sorted(zip(cosine_sim,individual_texts_train), reverse=True)]
    
    # print("Predicted Answer: ", individual_texts_train[:5])
    

            
    # ranks.append(1 / (preds.index(act_doc) + 1))

    # if act_doc == preds[0]:
    #     prec_at_1 += 1
    # if act_doc in preds[:5]:
    #     prec_at_5 += 1
    # if act_doc in preds[:10]:
    #     prec_at_10 += 1
    # if act_doc in preds[:50]:
    #     prec_at_50 += 1
    # if act_doc in preds[:100]:
    #     prec_at_100 += 1
    # if act_doc in preds[:500]:
    #     prec_at_500 += 1
    # sample_till_now += 1
    # if sample_till_now % 100 == 0:
    #     print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
    #     print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
    #           format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
    #                  prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
    #                  prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
    #                  sample_till_now))

/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/question_answering.py:313: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Context:  user: Hello, I forgot o update my address, can you help me with that?||agent: hi, you have to report any change of address to DMV within 10 days after moving. You should do this both for the address associated with your license and all the addresses associated with all your vehicles.||user: Can I do my DMV transactions online?||agent: Yes, you can sign up for MyDMV for all the online transactions needed.||
Actual Question:  You've got it. Another query about DMV. What happens if you seek a hearing with the DMV?
Short Question:  You've got it. Another query about DMV. What happens if you seek a hearing with the DMV?
roberta ans: ['A representative', 'A representative who s trained in the appeals process', 'Reschedule', 'A', 'Reschedule or withdraw', 'Fax your request to 844 - 678 - 8979', '844 - 678 - 8979', 'Reschedule or withdraw your Board hearing request \n\n', 'Reschedule or withdraw your Board hearing request', 'Reschedule or withdraw your Board hearing request \n\n']
Ac

/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/question_answering.py:313: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Context:  user: Hello, I forgot o update my address, can you help me with that?||agent: hi, you have to report any change of address to DMV within 10 days after moving. You should do this both for the address associated with your license and all the addresses associated with all your vehicles.||user: Can I do my DMV transactions online?||agent: Yes, you can sign up for MyDMV for all the online transactions needed.||user: You've got it. Another query about DMV. What happens if you seek a hearing with the DMV?||agent: the suspension is placed on hold pending the outcome of the hearing||
Actual Question:  Besides that, will I receive a notice?
Short Question:  Besides that, will I receive a notice?
roberta ans: ['no other company notified the DMV about new coverage', 'your insurance company notified the DMV that your insurance coverage ended', 'If you don t have insurance ,', 'If you don t have insurance', 'If you have insurance', 'The letter means your insurance company notified the DMV 

/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/question_answering.py:313: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

Context:  user: Can you help me with how to avoid the lapse of my insurance?||agent: DMV is always in contact with insurance companies to electronically monitor your insurance coverage and comply with the New York State requirements.||
Actual Question:  Can I just notify the United States Postal Service about a change of address or do I have to update this info with the DMV too?
Short Question:  Can you help me with how to avoid the lapse of my insurance?
roberta ans: ['If your registration has not expired , you must surrender your plates', 'Respond to DMV insurance letters and orders', 'See Respond to DMV insurance letters and orders', 'DMV insurance letters and orders', 'you must surrender your plates', 'Respond to DMV insurance letters', 'See Respond to DMV insurance letters', 'Respond to DMV insurance letters and orders [1]', 'Remember this simple rule: no insurance, no plates!', 'See Respond to DMV insurance letters and orders [1]']
Actual Answer:   It is not sufficient to only: w

/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/question_answering.py:313: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Context:  user: Can you help me with how to avoid the lapse of my insurance?||agent: DMV is always in contact with insurance companies to electronically monitor your insurance coverage and comply with the New York State requirements.||user: Can I just notify the United States Postal Service about a change of address or do I have to update this info with the DMV too?||agent: You have to keep your address up to date in the DMV. It is not enough to inform the US Postal Service.||
Actual Question:  I'd like to know if I will receive a Notice of suspension of Registration if I don't pay tolls?
Short Question:  Can I just notify the United States Postal Service about a change of address or do I have to update this info with the DMV too?
roberta ans: ['DMV', 'report a change of address to DMV', 'notify DMV', 'you must report a change of address to DMV', 'DMV', 'must report a change of address to DMV', 'By statute , you must report a change of address to DMV', 'tell the United States Postal Se

/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/question_answering.py:313: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)
/home/grads/r/rohan.chaudhury/miniconda3/envs/convoai/lib/python3.9/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


KeyboardInterrupt: 

In [90]:
from transformers import BartTokenizer, BartForQuestionAnswering
import torch

tokenizer2 = BartTokenizer.from_pretrained('valhalla/bart-large-finetuned-squadv1')
model2 = BartForQuestionAnswering.from_pretrained('valhalla/bart-large-finetuned-squadv1')

# question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
# encoding = tokenizer(question, text, return_tensors='pt')
# input_ids = encoding['input_ids']
# attention_mask = encoding['attention_mask']

# start_scores, end_scores = model(input_ids, attention_mask=attention_mask, output_attentions=False)[:2]

# all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
# answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
# answer = tokenizer.convert_tokens_to_ids(answer.split())
# answer = tokenizer.decode(answer)
# print(answer)

Downloading: 100%|██████████| 899k/899k [00:00<00:00, 3.71MB/s]
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 2.24MB/s]
Downloading: 100%|██████████| 150/150 [00:00<00:00, 79.1kB/s]
Downloading: 100%|██████████| 26.0/26.0 [00:00<00:00, 19.2kB/s]
Downloading: 100%|██████████| 1.43k/1.43k [00:00<00:00, 586kB/s]
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'LABEL_0', '1': 'LABEL_1'}. The number of labels wil be overwritten to 2.
Downloading: 100%|██████████| 1.63G/1.63G [00:43<00:00, 37.6MB/s]


In [91]:
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_answer=""
    l_question = qa_all[i]["short_question"]
    l_context = qa_all[i]["context"]
    for answers in qa_all[i]["answers"]:
        act_answer = act_answer + " " + answers["text"]
    accs, preds = predict_DR_TEIT([doc_sentence_test[i],
                                   doc_sentence_test[i-1],
                                   doc_sentence_test[i-2]],
                                   k=50,
                                   alpha=10)
    individual_texts_train = []
    # for pred in preds:
    #     for doc_idx2 in multidoc2dial_doc['doc_data'][pred]:
    #         individual_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
    #                                         [doc_idx2]['doc_text'].strip())
    all_text=[]
    short_question=doc_sentence_test[i]
    for pred in preds[:20]:
        # print(pred)
        # print(d_doc_data[pred]["spans"])
        all_text.append(d_doc_data[pred]["doc_text"])
        # k=1
        # string_int="1"
        # while(k==1):
        #     try:
        #         individual_texts_train.append(d_doc_data[pred]["spans"][string_int]["text_sp"].strip())
        #         string_int = str(int(string_int)+1)
        #     except:
        #         k=0
        # for t_spans in d_doc_data[pred]["spans"]:
        #     print(t_spans)
        #     individual_texts_train.append(d_doc_data[pred]["doc_text"][int(t_spans["start_sp"]):int(t_spans["end_sp"])].strip())
    
    # all_text = "\n".join(all_text)+"\n"+l_context
    for text in all_text:
        question, text = short_question, text[:1024]
        encoding = tokenizer2(question, text, return_tensors='pt')
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']

        start_scores, end_scores = model2(input_ids, attention_mask=attention_mask, output_attentions=False)[:2]

        all_tokens = tokenizer2.convert_ids_to_tokens(input_ids[0])
        answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
        answer = tokenizer2.convert_tokens_to_ids(answer.split())
        answer = tokenizer2.decode(answer)
        print("bart_answer:", answer)
    # QA_input = {
    #       'question': long_question,
    #       'context': all_text
    #       }
    
    # res = nlp(QA_input, max_answer_len=100,  handle_impossible_answer=True, max_question_len=100, allow_long_answers=True, align_to_words=True)
    # res2= nlp(QA_input2, max_answer_len=100, handle_impossible_answer=True, max_question_len=100, allow_long_answers=True, align_to_words=True)
    
    prel_ans=answer
    # prel_ans2=res2["answer"]
    # print("Context: ", l_context+ "||")
    print("Actual Question: ", l_question)
    print("Short Question: ", short_question)
    # print("bart ans:", prel_ans)
    print("Actual Answer: ", act_answer)
    print("\n\n")
    # individual_texts_train = list(set(individual_texts_train))
    # cosine_sim=[]
    # query_embedding = get_embeddings(prel_ans).reshape(1,-1)
    # for i in range(len(individual_texts_train)):
    #     cosine_sim.append(cosine_similarity(query_embedding, get_embeddings(individual_texts_train[i]).reshape(1,-1))[0][0])
    # individual_texts_train = [x for _,x in sorted(zip(cosine_sim,individual_texts_train), reverse=True)]
    
    # print("Predicted Answer: ", individual_texts_train[:5])
    

            
    # ranks.append(1 / (preds.index(act_doc) + 1))

    # if act_doc == preds[0]:
    #     prec_at_1 += 1
    # if act_doc in preds[:5]:
    #     prec_at_5 += 1
    # if act_doc in preds[:10]:
    #     prec_at_10 += 1
    # if act_doc in preds[:50]:
    #     prec_at_50 += 1
    # if act_doc in preds[:100]:
    #     prec_at_100 += 1
    # if act_doc in preds[:500]:
    #     prec_at_500 += 1
    # sample_till_now += 1
    # if sample_till_now % 100 == 0:
    #     print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
    #     print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
    #           format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
    #                  prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
    #                  prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
    #                  sample_till_now))

bart_answer:  added to your appeal file
bart_answer:  A video conference hearing will take place at a VA location near you
bart_answer:  A video conference hearing will take place at a VA location near you
bart_answer:  Forgetting to Update Address 
By statute, you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current, you will miss a suspension order and may be charged with operating an unregistered vehicle and/or aggravated unlicensed operation, both misdemeanors.
bart_answer:  contact your insurance company immediately
bart_answer:  Enter a valid email address, and re - enter the email address to verify it. Th

KeyboardInterrupt: 